# Overview

In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

%matplotlib inline

## Data Import

In [ ]:
symbols=["SPY", "AAPL", "MSFT", "AMZN", "FB", "GOOGL", "GOOG", "BRK.B", "JNJ", "JPM", "V"] #big 10 symbols to collect
pricetype = "close"
analysis_date_start = "2019-01-01"
analysis_date_end = "2020-02-28"

In [ ]:
for symbol in symbols:
    filepath = "../data/{}_daily01.csv".format(symbol)
    stock_df = pd.read_csv(filepath)
    stock_df = stock_df[(stock_df.to_datetime > analysis_date_start) & (stock_df.to_datetime < analysis_date_end)]["close"]
    plot_title = "{} ACF and PACF".format(symbol)
    fig, ax = plt.subplots(1,2, figsize=(12,8))
    plot_acf(stock_df.values, ax=ax[0]);
    plot_pacf(stock_df.values, ax=ax[1]);
    fig.suptitle(plot_title, size=18)

In [ ]:
def ARIMA_gridsearch(p_max:int, d_max:int, q_max:int, data, prediction_length):
    return_list = [] #initialize return list
    train_data_idx = len(data) - prediction_length
    train = data[:train_data_idx] #slice train and test data
    test = data[train_data_idx:]
    p_range = range(1, p_max) #set ARIMA ranges
    d_range = range(0, d_max)
    q_range = range(1, q_max)
    for p in p_range: #loop through ARIMA ranges
        for d in d_range:
            for q in q_range:
                model = ARIMA(train, order=(p,d,q)) #fit and predict
                arima_model = model.fit()
                model_prediction = arima_model.forecast(steps=prediction_length)
                warning = arima_model.mle_retvals["warnflag"]
                convergence = arima_model.mle_retvals["converged"]
                rmse_result = np.sqrt(mean_squared_error(test, model_prediction)) #get rmse
                return_list.append([p,d,q,convergence, warning, rmse_result])
    return sorted(return_list, key=lambda x: (x[-1], x[-2], (x[-3]*-1)))